In [1]:
!pip install pyspark

     |████████████████████████████████| 281.4 MB 30 kB/s 
     |████████████████████████████████| 198 kB 43.2 MB/s 
  Created wheel for pyspark: filename=pyspark-3.2.1-py2.py3-none-any.whl size=281853642 sha256=e8b2fdec6ef02b832a08798564ccf7a92971a707895eed7475fdb0bb75f5d1af
  Stored in directory: /root/.cache/pip/wheels/9f/f5/07/7cd8017084dce4e93e84e92efd1e1d5334db05f2e83bcef74f
Successfully built pyspark


In [2]:
from pyspark.sql import SparkSession

In [3]:
session = SparkSession.builder.appName('regression').master('local').getOrCreate()

In [4]:
data = session.read.csv('/content/energy_cooling.csv',header = True, inferSchema = True)

In [5]:
data.show(5)

+----+-----+-----+------+---+---+---+---+-----+
|  X1|   X2|   X3|    X4| X5| X6| X7| X8|   Y2|
+----+-----+-----+------+---+---+---+---+-----+
|0.98|514.5|294.0|110.25|7.0|  2|0.0|  0|21.33|
|0.98|514.5|294.0|110.25|7.0|  3|0.0|  0|21.33|
|0.98|514.5|294.0|110.25|7.0|  4|0.0|  0|21.33|
|0.98|514.5|294.0|110.25|7.0|  5|0.0|  0|21.33|
| 0.9|563.5|318.5| 122.5|7.0|  2|0.0|  0|28.28|
+----+-----+-----+------+---+---+---+---+-----+
only showing top 5 rows



In [6]:
data.printSchema()

root
 |-- X1: double (nullable = true)
 |-- X2: double (nullable = true)
 |-- X3: double (nullable = true)
 |-- X4: double (nullable = true)
 |-- X5: double (nullable = true)
 |-- X6: integer (nullable = true)
 |-- X7: double (nullable = true)
 |-- X8: integer (nullable = true)
 |-- Y2: double (nullable = true)



In [7]:
from pyspark.ml.feature import VectorAssembler

In [8]:
vec = VectorAssembler(inputCols = ['X1','X2','X3','X4','X5','X6','X7','X8'], outputCol = "all_features")

In [9]:
from pyspark.ml.regression import LinearRegression
#from pyspark.ml.regression import DecisiontreeRegressor 
# can also be used

In [10]:
model = LinearRegression(featuresCol = "all_features", labelCol = "Y2")

In [13]:
from pyspark.ml import Pipeline
mypipeline = Pipeline(stages = [vec, model])

In [14]:
training, test = data.randomSplit([0.7,0.3])
model = mypipeline.fit(training)
result = model.transform(test)
result.show(5)

+----+-----+-----+-----+---+---+----+---+-----+--------------------+------------------+
|  X1|   X2|   X3|   X4| X5| X6|  X7| X8|   Y2|        all_features|        prediction|
+----+-----+-----+-----+---+---+----+---+-----+--------------------+------------------+
|0.62|808.5|367.5|220.5|3.5|  2| 0.0|  0|12.07|[0.62,808.5,367.5...|13.827704731451632|
|0.62|808.5|367.5|220.5|3.5|  2| 0.1|  2|14.37|[0.62,808.5,367.5...|15.481128152791129|
|0.62|808.5|367.5|220.5|3.5|  2| 0.1|  5|14.24|[0.62,808.5,367.5...|15.638008942644916|
|0.62|808.5|367.5|220.5|3.5|  2|0.25|  2|15.32|[0.62,808.5,367.5...| 17.80438249494658|
|0.62|808.5|367.5|220.5|3.5|  2| 0.4|  1| 17.2|[0.62,808.5,367.5...|20.075343240484102|
+----+-----+-----+-----+---+---+----+---+-----+--------------------+------------------+
only showing top 5 rows



In [15]:
from pyspark.ml.evaluation import RegressionEvaluator



In [18]:
regeval = RegressionEvaluator(predictionCol = 'prediction', labelCol ='Y2', metricName = 'rmse' )
regeval.evaluate(result)

3.219980309198298